Load API Keys

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

Load Contents from Brillar Bank

In [2]:
from langchain_community.document_loaders import WebBaseLoader
from helpers.test_data import get_urls

loader = WebBaseLoader(get_urls())
data = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


Split the Loaded Data into Chunks

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=20)
texts = text_splitter.split_documents(data)

Get Embedding Model

In [4]:
from langchain_openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings(model="text-embedding-3-large", dimensions=256)

Instantiate Elasticsearch Store

In [5]:
from langchain_elasticsearch import ElasticsearchStore, SparseVectorStrategy, BM25Strategy
import os

es_cloud_id = os.getenv("ELASTIC_CLOUD_ID")
es_api_key = os.getenv("ELASTIC_API_KEY")

sparse_vector_store = ElasticsearchStore.from_documents(
    documents=texts,
    es_cloud_id=es_cloud_id,
    es_api_key=es_api_key,
    index_name="just_sparse",
    strategy=SparseVectorStrategy(model_id=".elser_model_2"),
    bulk_kwargs={
      "request_timeout": 60
    }
)

Get LLM

In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")

Get Retriever

In [7]:
retriever = sparse_vector_store.as_retriever(search_kwargs={"k": 5})

Create Chain

In [8]:
from helpers.conversation_retrieval_chain import create_conversational_retrieval_chain, invoke_conversational_retrieval_chain

chain = create_conversational_retrieval_chain(llm, retriever)

Invoke Chain

In [9]:
question = "what are the interest rates for fixed deposit?"
result = invoke_conversational_retrieval_chain(chain, question)
result

{'input': 'what are the interest rates for fixed deposit?',
 'chat_history': [],
 'context': [Document(metadata={'language': 'en', 'source': 'https://win066.wixsite.com/brillar-bank/junior-fixed-deposit', 'title': 'Junior Fixed Dep... | Brillar Bank'}, page_content="Parent or legal guardian's MyKad/passport.Terms and Conditions applyMember of PIDM. Protected by PIDM up to RM250,000 for each depositor.Interest RatesTenureInterest Rates1 month2.15%2 - 3 months2.25%4 - 5 months2.30%6 months2.30%7 - 11 months2.35%12 - 60 months2.50%Fees and ChargesFees & Charges DescriptionFees / Charges\n(subject to Government Tax, if applicable)Cheque Related :Dishonoured Inward Return Cheques due to:\nPost-dated reason onlyRM10.00 per chequeChanges to Account Related :\n\nChange to operating mandate\n\n\nAddition of joint accountholder\n\nRM10.00 per account\n\nRM10.00 per account\n\n(No charge for Priority Banking customers)Service Charge Related :Average balance of the Deposit Accounts fall below the 